In [1]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

In [ ]:
class MC(object):

    def __init__(self):
        self.player_state_value = {}
        self.player_states = []
        self.player_win = 0     # Number of player wins
        self.player_draw = 0    # Number of player ties

    # Get a card - Assumes infinte number of decks mixed together
    @staticmethod
    def giveCard():
        cards = [1,2,3,4,5,6,7,8,9,10,10,10]
        return np.random.choice(cards)
    

    def dealerLogic(self, val, ace):
        if val > 21:
            if ace:
                val -= 10
                ace = False
            else:
                return val, ace, True    
        # Assuming dealer stands on hard or soft 17, can be changed later
        if val >= 17:
            return val, ace, True
        return self.aceHelper(val,ace)

    
    def playerLogic(self, val, ace):
        if val > 21:
            # Calculate ace value if player sum is too high, return loss if no ace
            if ace:
                val -= 10
                ace = False
            else:
                return val, ace, True
            
        # Don't hit 20- this can be changed later for testing
        if val >= 20:
            return val, ace, True
        return self.aceHelper(val,ace)
    

    def aceHelper(self, val, ace):
        card = self.giveCard()
        if card == 1:
            if val + 11 > 21:
                # Card must be hard ace (1)
                return val + 1, ace, False
            else:
                # Card can be a soft ace (11)
                return val + 11, True, False
        else:
            return val+card, ace, False
            

    def _Reinforce(self, player_val, dealer_val, is_end=True):
        if is_end:
            # Evaluate only if player is done playing
            
            last_state = self.player_states[-1]
            if player_val > 21:
                # if dealer_val > 21:
                    # self.player_draw += 1
                #else:
                    #self.player_state_value[last_state] -= 1

                ### I think there was an issue here- Player should lose because they play before dealer
                self.player_state_value[last_state] -= 1

            else:
                # Dealer busted and player has valid sum
                if dealer_val > 21:
                    self.player_state_value[last_state] += 1
                    self.player_win += 1
                else:
                    # Dealer has higher score- lose
                    if player_val < dealer_val:
                        self.player_state_value[last_state] -= 1

                    # Player has higher score- win
                    elif player_val > dealer_val:
                        self.player_state_value[last_state] += 1
                        self.player_win += 1
                    else:
                        # Draw- player/ dealer tied
                        self.player_draw += 1
    